In [1]:
import os
import pandas as pd
import requests
import bs4
import lxml
import functions.web_scrapper_functions as scrpr
import mysql.connector
import json
import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import logging
import warnings
warnings.filterwarnings("ignore")

logging.basicConfig(filename='Parfum AE Web Scrapper Log.log', level=logging.INFO,
format='%(asctime)s:%(levelname)s:%(message)s')

In [2]:
frag_type_dict = scrpr.get_frag_types()
print("Available fragrance types: \n", frag_type_dict.keys())

Available fragrance types: 
 dict_keys(['Body Mist', 'Car Perfume Diffuser', 'Perfume Oils', 'Our Creations', 'Dahn Al Oud', 'Interior Perfume', 'Musk Oil', 'Perfume Reed Diffuser'])


In [4]:
prod_list_dict = {}
start_time = datetime.datetime.now()

for key, val in frag_type_dict.items():
    try:
        logging.info('Currently scrapping ', key)
        prod_list_dict[key] = scrpr.prod_list(val)
        logging.info('Number of products: ', len(prod_list_dict[key]))
    except:
        logging.error("Error in ", key)
end_time = datetime.datetime.now()
time_taken = end_time - start_time
logging.info(f"Time taken to retrieve list of all products: {time_taken}")

--- Logging error ---
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 1081, in emit
    msg = self.format(record)
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 925, in format
    return fmt.format(record)
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 664, in format
    record.message = record.getMessage()
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\users\shahe\onedrive\documents\projects\project 4 - parfum.ae analysis\parfumaeproject\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  Fi

In [5]:
start_time = datetime.datetime.now()
products = scrpr.prod_info(prod_list_dict)
end_time = datetime.datetime.now()
time_taken = end_time - start_time
logging.info(f"Time taken to retrieve information for each product: {time_taken}")

In [11]:
products

{'Oud Cambodi Synthetic': {'id': '1094',
  'name': 'Oud Cambodi Synthetic',
  'stars': 5,
  'reviews': [['Sabrina ',
    'Actually I like it this product good job team Parfum.ae',
    5],
   ['Abden', 'Good Product', 4],
   ['noor  Almazrouei', 'Really its very good and good for summer! ', 5],
   ['liyaan',
    "This is exactly what I was looking for. Crisp, clean, woody, bitter and medicinal with the slightly cool mintiness. This has set the bar for me personally and is the scent of kyara I\\'ve heard described and knew I had to try. development on the skin is divine. Probably best not to use before bed, it\\'s strong! No idea how the price is so reasonable, baffles me. I know I\\'ll be searching for a long time for something cultivated and dirt cheap that smells like this so I can slather it on daily. ",
    5],
   ['Rufat Abdullayev',
    'A fabulous Oud oil, thick like honey, among the best agarwoody perfumes ',
    5],
   ['AbdRakhman ', 'Strong citrus in the opening and smokey va

In [34]:
reviews

{'Sabrina ': 'Actually I like it this product good job team Parfum.ae',
 'Abden': 'Good Product',
 'noor  Almazrouei': 'Really its very good and good for summer! ',
 'liyaan': "This is exactly what I was looking for. Crisp, clean, woody, bitter and medicinal with the slightly cool mintiness. This has set the bar for me personally and is the scent of kyara I\\'ve heard described and knew I had to try. development on the skin is divine. Probably best not to use before bed, it\\'s strong! No idea how the price is so reasonable, baffles me. I know I\\'ll be searching for a long time for something cultivated and dirt cheap that smells like this so I can slather it on daily. ",
 'Rufat Abdullayev': 'A fabulous Oud oil, thick like honey, among the best agarwoody perfumes '}

In [36]:
product_reviews('https://www.parfum.ae/oud-cambodi-synthetic/')

[{'Sabrina ': 'Actually I like it this product good job team Parfum.ae'},
 {'Abden': 'Good Product'},
 {'noor  Almazrouei': 'Really its very good and good for summer! '},
 {'liyaan': "This is exactly what I was looking for. Crisp, clean, woody, bitter and medicinal with the slightly cool mintiness. This has set the bar for me personally and is the scent of kyara I\\'ve heard described and knew I had to try. development on the skin is divine. Probably best not to use before bed, it\\'s strong! No idea how the price is so reasonable, baffles me. I know I\\'ll be searching for a long time for something cultivated and dirt cheap that smells like this so I can slather it on daily. "},
 {'Rufat Abdullayev': 'A fabulous Oud oil, thick like honey, among the best agarwoody perfumes '}]

In [ ]:
conn = mysql.connector.connect(
    host = "localhost",
    user = os.environ.get("MYSQL_USERNAME"),
    password = os.environ.get("MYSQL_PASSWORD"),
    database = "parfumae_products"
)

cursor = conn.cursor()

In [ ]:

try:
    for key, val in frag_type_dict.items():
        serialized_value = json.dumps(val)

        insert_query = "INSERT INTO fragrance_types (name, url) VALUES (%s, %s)"
        cursor.execute(insert_query, (key, serialized_value))
    
    conn.commit()

except mysql.connector.Error as err:
    print(f"Error: {err}")
    conn.rollback()

cursor.close()
conn.close()